In [1]:
import cv2
import numpy as np
import os
import tensorflow as tf

from sklearn.model_selection import train_test_split

In [9]:
IMG_WIDTH, IMG_HEIGHT = 28, 28
NUM_CATEGORIES = 10
MODEL_FILE = 'handwritten.keras'
DATASET_DIR = 'dataset'

In [3]:
def get_image_as_array(image_path):
    arr = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)
    if arr.shape[2] == 4:     # we have an alpha channel
        a1 = ~arr[:,:,3]        # extract and invert that alpha
        arr = cv2.add(cv2.merge([a1,a1,a1,a1]), arr)   # add up values (with clipping)
        arr = cv2.cvtColor(arr, cv2.COLOR_RGBA2RGB)
    
    return cv2.resize(arr, (IMG_WIDTH, IMG_HEIGHT))


def load_data(data_dir):
    images = []
    labels = []
    for directory in os.listdir(data_dir):
        if directory.startswith('.'):
            continue

        for img in os.listdir(os.path.join(data_dir, directory)):
            if img.startswith('.'):
                continue
            
            images.append(get_image_as_array(os.path.join(data_dir, directory, img)))
            labels.append(int(directory))

    return images, labels

In [49]:
def get_model():
    data_augmentation = tf.keras.Sequential([
      tf.keras.layers.RandomRotation(0.3),
    ])
    model = tf.keras.models.Sequential([
        tf.keras.Input(shape=(IMG_WIDTH, IMG_HEIGHT, 3)),
        data_augmentation,
        tf.keras.layers.Conv2D(26, (4, 4), activation="relu"),

        tf.keras.layers.MaxPooling2D(pool_size=(3, 3)),
        tf.keras.layers.Flatten(),

        tf.keras.layers.Dense(128, activation="relu"),
        tf.keras.layers.Dense(128, activation="relu"),
        tf.keras.layers.Dropout(0.3),

        tf.keras.layers.Dense(NUM_CATEGORIES, activation="softmax")
    ])

    model.compile(
        optimizer="adam",
        loss="categorical_crossentropy",
        metrics=["accuracy"]
    )
    return model

In [50]:
def train_model():
    images, labels = load_data(DATASET_DIR)

    labels = tf.keras.utils.to_categorical(labels)
    x_train, x_test, y_train, y_test = train_test_split(np.array(images), np.array(labels), test_size=0.3)

    model = get_model()
    model.fit(x_train, y_train, epochs=10)
    model.evaluate(x_test,  y_test, verbose=2)

    model.save(MODEL_FILE)
    print(f"Model saved to {MODEL_FILE}")

In [47]:
def predict(image_path) -> int:
    image_arr = get_image_as_array(image_path)
    model = tf.keras.models.load_model(MODEL_FILE)
    predictions = model.predict(np.expand_dims(image_arr, axis=0))
    # print(predictions)
    return np.argmax(predictions, axis=-1)[0]

In [51]:
train_model()

Epoch 1/10
1532/1532 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.2252 - loss: 4.0785
Epoch 2/10
1532/1532 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.6323 - loss: 1.1600
Epoch 3/10
1532/1532 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.7069 - loss: 0.9190
Epoch 4/10
1532/1532 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.7395 - loss: 0.8277
Epoch 5/10
1532/1532 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.7697 - loss: 0.7312
Epoch 6/10
1532/1532 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.7865 - loss: 0.6954
Epoch 7/10
1532/1532 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.7946 - loss: 0.6599
Epoch 8/10
1532/1532 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.8013 - loss: 0.6384
Epoch 9/10
1532/1532 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.8084 - loss: 0.6188
Epoch 10/10
1532/1532 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.8125 - loss: 0.6155
657/657 - 1s - 848us/step - accuracy: 0.8578 - loss: 0.4650
Model saved to handwritten2.keras


In [52]:
downloads = '/Users/dmytroberehovets/Downloads'
for f in os.listdir(downloads):
    if f.startswith('.'):
        continue
    
    print(f, predict(os.path.join(downloads, f)))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
71.png 4
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
8.png 8
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
7r.png 7
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
81.png 8
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
5.png 5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
7.png 7
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
2.png 2
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
5r.png 5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1.png 1
